# TFT
https://arxiv.org/abs/1912.09363

## Imports:

In [16]:
import pandas as pd
from darts import TimeSeries
from pathlib import Path
from darts.models.forecasting.tft_model import TFTModel
from darts.utils.model_selection import train_test_split
import sys
from sklearn.preprocessing import StandardScaler
sys.path.append('../Helper/')
from dataPreprocessing import add_time_features, rank_features_ccf, get_untransformed_exog
from darts.dataprocessing.transformers import Scaler
from torch.nn import MSELoss
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from torchmetrics import MeanSquaredError

# Data Preprocessing:

In [2]:
cwd=Path.cwd()
data= pd.read_csv(cwd.parent.parent / 'Data' /'Train'/'train1990s.csv',parse_dates=[0])
data= get_untransformed_exog(data)
ranked_cols= rank_features_ccf(data.drop('observation_date',axis=1))
data_ranked= data[ranked_cols[:5]]
data_ranked['observation_date']= data['observation_date']
data_ranked['fred_PCEPI']= data['fred_PCEPI']
add_time_features(data_ranked)
display(data_ranked)

C:\Users\kevin\AppData\Local\Temp\ipykernel_6424\571883820.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data= pd.read_csv(cwd.parent.parent / 'Data' /'Train'/'train1990s.csv',parse_dates=[0])
C:\Users\kevin\AppData\Local\Temp\ipykernel_6424\571883820.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ranked['observation_date']= data['observation_date']
C:\Users\kevin\AppData\Local\Temp\ipykernel_6424\571883820.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

,fred_PCUOMFGOMFG,fred_PPIACO,fred_APU000074714,BrentOil_Open,CrudeOilWTI_Open,observation_date,fred_PCEPI,year,month,quarter
0,112.700,114.900,1.042,20.20,21.81,1990-01-01,58.553,1990,1,1
1,112.200,114.400,1.037,19.90,22.61,1990-02-01,58.811,1990,2,1
2,112.300,114.200,1.023,19.48,21.73,1990-03-01,59.033,1990,3,1
3,112.600,114.100,1.044,18.49,20.20,1990-04-01,59.157,1990,4,2
4,113.100,114.600,1.061,17.20,18.46,1990-05-01,59.290,1990,5,2
...,...,...,...,...,...,...,...,...,...,...
403,251.093,257.680,3.955,84.78,81.64,2023-08-01,120.965,2023,8,3
404,252.368,258.934,3.988,86.22,83.63,2023-09-01,121.387,2023,9,3
405,249.119,255.192,3.782,90.48,90.88,2023-10-01,121.421,2023,10,4
406,246.497,252.856,3.500,84.78,81.31,2023-11-01,121.415,2023,11,4


## Constants:

In [3]:
scaler1=Scaler(StandardScaler())
scaler2=Scaler(StandardScaler())
scaler3=Scaler(StandardScaler())
INPUT_SIZE=12
HORIZON=12
HIDDEN_SIZE=4

VALID_SIZE=12

In [4]:
def darts_get_inference_set(past_covariates,future_covariates,target, valid_size, input_size, time_col="observation_date", freq= "MS"):
    X_train_future= TimeSeries.from_dataframe(future_covariates.iloc[:-valid_size,:],time_col=time_col,freq=freq)
    X_train_past= TimeSeries.from_dataframe(past_covariates.iloc[:-valid_size,:],time_col=time_col,freq=freq)
    y_train= TimeSeries.from_dataframe(target.iloc[:-valid_size,:],time_col=time_col,freq=freq)

    X_valid_future= TimeSeries.from_dataframe(future_covariates.iloc[-valid_size-input_size:,:],time_col=time_col,freq=freq)
    X_valid_past= TimeSeries.from_dataframe(past_covariates.iloc[-valid_size-input_size:,:],time_col=time_col,freq=freq)
    y_valid= TimeSeries.from_dataframe(target.iloc[-valid_size-input_size:,:],time_col=time_col,freq=freq)

    return X_train_past, X_valid_past, X_train_future, X_valid_future, y_train, y_valid


## Create TimeSeries objects:

In [5]:
future_covariates= data_ranked[["observation_date","year","month", "quarter"]]
past_covariates= data_ranked.drop(["fred_PCEPI","year","month", "quarter"],axis=1)
target= data_ranked[["observation_date","fred_PCEPI"]]

In [6]:

X_train_past, X_valid_past, X_train_future, X_valid_future, y_train, y_valid=darts_get_inference_set(past_covariates,future_covariates,target, valid_size=VALID_SIZE,input_size=INPUT_SIZE)

## Feature Scaling:

In [7]:

X_train= scaler1.fit_transform(X_train_past)
X_train_future= scaler2.fit_transform(X_train_future)
y_train= scaler3.fit_transform(y_train)


X_valid=scaler1.transform(X_valid_past)
X_valid_future= scaler2.transform(X_valid_future)
y_valid=scaler3.transform(y_valid)


# Training:
To optimize:
* number of features
* hidden_size [1,2,4,8,16,32]
* lstm layers [1,2,4]
* num_attention_heads [1,2,4,8]
* lr [1e-2,1e-3,1e-4,1e-5]


In [8]:
valid_metric= MeanSquaredError()

In [ ]:
early_stopper = EarlyStopping(
    monitor="val_MeanSquaredError",
    patience=50,
    min_delta=1e-5,
    mode='min',
)
model_checkpoint= ModelCheckpoint('.', 'TFT_best_weights', monitor="val_MeanSquaredError",save_top_k=1, mode="min", save_weights_only=True)

In [ ]:
model = TFTModel(input_chunk_length=INPUT_SIZE,output_chunk_length=HORIZON,hidden_size=HIDDEN_SIZE,
                 batch_size=256, use_static_covariates=False, add_relative_index=True, likelihood=None, loss_fn = MSELoss(),
                 torch_metrics=valid_metric, save_checkpoints=True, random_state=42, 
                 ,pl_trainer_kwargs={
                                    "accelerator":"gpu",
                                    "devices": -1,
                                    "callbacks": [early_stopper, model_checkpoint]
                                    })

model.fit(y_train,past_covariates=X_train_past,future_covariates=X_train_future,val_series=y_valid, val_past_covariates=X_valid_past,val_future_covariates=X_valid_future, epochs=10000)#,past_covariates=X_train,future_covariates=X_train_future,

2025-03-31 15:19:50,013 - INFO - Train dataset contains 373 samples.
2025-03-31 15:19:50,043 - INFO - Time series values are 64-bits; casting model to float64.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory H:\My Drive\UniversityOfLeeds\CS\Year4\GroupProject\repo\COMP5530M-Group-Project-Inflation-Forecasting\Training\TFT exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                              | Type                             | Params | Mode 
------------------------------------------------------------------------------------------------
0  | criterion                         | MSELoss                          | 0      | train
1  | train_criterion                   | MSELoss                          | 0      | train
2  | val_criterion     

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\core\module.py:512: You called `self.log('val_MeanSquaredError', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

TFTModel(output_chunk_shift=0, hidden_size=4, lstm_layers=1, num_attention_heads=4, full_attention=False, feed_forward=GatedResidualNetwork, dropout=0.1, hidden_continuous_size=8, categorical_embedding_sizes=None, add_relative_index=True, loss_fn=MSELoss(), likelihood=None, norm_type=LayerNorm, use_static_covariates=False, input_chunk_length=12, output_chunk_length=12, batch_size=256, torch_metrics=MeanSquaredError(), save_checkpoints=True, random_state=42, pl_trainer_kwargs={'accelerator': 'gpu', 'devices': -1, 'callbacks': [<pytorch_lightning.callbacks.early_stopping.EarlyStopping object at 0x000001A935047A70>, <pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x000001A9350473E0>]})

In [36]:
import torch
torch.save(torch.load(model_checkpoint.best_model_path, weights_only=False)["state_dict"],'sate_dict.pt')

In [52]:
model.load('sate_dict.pt')

2025-03-31 15:54:09,540 - WARNING - Model was loaded without weights since no PyTorch LightningModule checkpoint ('.ckpt') could be found at sate_dict.pt.ckpt. Please call `fit()` before calling `predict()`.


OrderedDict([('encoder_vsn.flattened_grn.resample_norm.mask',
              tensor([-0.1445, -0.1287,  0.0223,  0.0526,  0.0810, -0.0081,  0.0868,  0.0858,
                      -0.0046, -0.1681], device='cuda:0', dtype=torch.float64)),
             ('encoder_vsn.flattened_grn.resample_norm.norm.weight',
              tensor([1.0265, 0.9073, 1.0262, 1.0137, 1.0172, 0.9765, 1.1089, 0.9725, 0.9751,
                      0.9891], device='cuda:0', dtype=torch.float64)),
             ('encoder_vsn.flattened_grn.resample_norm.norm.bias',
              tensor([ 0.0294, -0.0935, -0.0263,  0.0112,  0.0131, -0.0333,  0.0903, -0.0278,
                      -0.0250, -0.0086], device='cuda:0', dtype=torch.float64)),
             ('encoder_vsn.flattened_grn.fc1.weight',
              tensor([[-0.1193,  0.2489, -0.0962, -0.0032,  0.1459,  0.0022,  0.3800,  0.1196,
                       -0.2267, -0.0051,  0.1632, -0.0320, -0.0422,  0.0763, -0.0788,  0.1249,
                       -0.0510, -0.0295, -0

In [ ]:
scaler3.inverse_transform(model.predict(12,series=y_train,past_covariates=X_valid_past, future_covariates=X_valid_future)).to_dataframe()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

component,fred_PCEPI
observation_date,
2023-01-01,114.369289
2023-02-01,115.009689
2023-03-01,115.598042
2023-04-01,116.296947
2023-05-01,116.593628
2023-06-01,116.754337
2023-07-01,116.854588
2023-08-01,116.869020
2023-09-01,116.869794
